In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras_vggface.vggface import VGGFace


Using TensorFlow backend.


In [2]:
#custom parameters
nb_class = 5
hidden_dim = 512

vgg_model = VGGFace(model='vgg16',include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
#vgg_model.load_weights("C:\\Users\\isaac yang\\.keras\\models\\vggface\\rcmalli_vggface_tf_notop_vgg16.h5")

In [4]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [5]:
vgg_model.trainable = False

In [6]:
classifier_model=Sequential()


In [7]:
classifier_model.add(vgg_model)
classifier_model.add(Flatten())
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
# classifier_model.add(Dropout(0.3))
# classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
# classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(nb_class, activation = 'softmax'))


In [8]:
model = classifier_model

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_vgg16 (Model)        (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125445    
Total params: 14,840,133
Trainable params: 125,445
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
from keras.preprocessing.image import ImageDataGenerator
img_width = img_height = 224
channels = 3
datagen = ImageDataGenerator(
    featurewise_center=True,
    zca_whitening=True,
    zca_epsilon=1e-06,
    #rotation_range=40, #I'm thinking if a person tilts his head it will be 40 degrees maximum
    #width_shift_range=0.5, #If up to half the person's face is visible width-wise
    #height_shift_range=0.5, #If up to half the person's face is visible height-wise
    #brightness_range=[0.2,2.0], #20% or 200% brightness
    #shear_range=10.0, #putting it very small here because I can't find any tutorials with exact values of good shear range angles
    #zoom_range=[0.3,3], #zooming up to 3 times and minimizing up to 3 times
    #channel_shift_range= 200, #Increases color variation in case someone's face suddenly becomes very tanned, can check the rgb values table
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255,
    validation_split=0.3,
)



In [10]:
batch_size = 16
train_it = datagen.flow_from_directory('data/train2/',
                                       target_size = (img_height, img_width),
                                       class_mode='categorical', 
                                       batch_size=batch_size)

val_it = datagen.flow_from_directory('data/val2/', 
                                    class_mode='categorical',
                                    target_size = (img_height, img_width),
                                    batch_size=batch_size)


Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [13]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
filepath="C:\\development\\NDSC_junior\\SRP\\weights\\vgg16\\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

# earlystop = EarlyStopping(monitor = 'val_loss',
#                          min_delta = 0,
#                          patience = 3,
#                          verbose = 1,
#                          restore_best_weights = True)
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')

nb_train_samples = 64
nb_validation_samples = 16

epochs = 10
steps_per_epoch = nb_train_samples/batch_size
validation_steps = nb_validation_samples/batch_size

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, min_delta = 0.0001, patience=3, min_lr=0, cooldown = 0)
callbacks = [checkpoint, reduce_lr]


model.compile(loss = 'categorical_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [14]:
model.fit_generator(train_it, 
                    validation_data = val_it, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_steps=validation_steps, 
                    epochs = epochs, 
                    callbacks= callbacks)

Epoch 1/10
4/4 [==============================] - 48s 12s/step - loss: 1.5040 - acc: 0.3125 - val_loss: 1.0947 - val_acc: 0.7778

Epoch 00001: val_loss improved from inf to 1.09473, saving model to C:\development\NDSC_junior\SRP\weights\vgg16\weights-improvement-01-1.09.hdf5
Epoch 2/10
4/4 [==============================] - 48s 12s/step - loss: 1.0612 - acc: 0.7805 - val_loss: 0.8466 - val_acc: 0.8125

Epoch 00002: val_loss improved from 1.09473 to 0.84656, saving model to C:\development\NDSC_junior\SRP\weights\vgg16\weights-improvement-02-0.85.hdf5
Epoch 3/10
4/4 [==============================] - 35s 9s/step - loss: 0.6810 - acc: 0.9842 - val_loss: 0.5893 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.84656 to 0.58926, saving model to C:\development\NDSC_junior\SRP\weights\vgg16\weights-improvement-03-0.59.hdf5
Epoch 4/10
4/4 [==============================] - 43s 11s/step - loss: 0.4635 - acc: 0.9844 - val_loss: 0.3203 - val_acc: 1.0000

Epoch 00004: val_loss improved from

In [16]:
model.save("C:\\development\\NDSC_junior\\\SRP\models\\" + "vgg16")


In [19]:
print(train_it.class_indices)

{'ben_afflek': 0, 'elton_john': 1, 'jerry_seinfeld': 2, 'madonna': 3, 'mindy_kaling': 4}
